**Problem 4.5**
**a)**
When coming up with our priors we like to assume ignorance. Using Jeffrey's prior makes sense because the rate of dissociation can also be represented by its reciprical. It is a rate constant and has no units. 

**b)** First we import our workhorses and load in our data set. 

In [ ]:
import collections
import itertools

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats as st

# Numerical differentiation packages
import numdifftools as ndt

# Our main MCMC package
import emcee

# Import pyplot for plotting
import matplotlib.pyplot as plt

# Seaborn, useful for graphics
import seaborn as sns

# Corner is useful for displaying MCMC results
import corner

%matplotlib inline

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2,
      'axes.labelsize': 18,
      'axes.titlesize': 18,
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

# Load data
df = pd.read_csv('./data/rasson_et_al/', comment = '#')